In [ ]:
# nbi:hide_in
%matplotlib inline

from ipywidgets import interact, interactive, fixed
from IPython.display import clear_output, display, HTML

import math
# import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import cnames
from matplotlib import animation
import matplotlib.gridspec as gridspec

from scipy import integrate

plt.style.use('seaborn-darkgrid')
plt.rcParams.update({"axes.edgecolor" : "w"})

# Compartmental models in epidemiology

In [ ]:
# nbi:hide_in
def epid_model(comps_labels, inf_index, diff_eqs, 
    calc_mode, plot_type, DELTA_TIME, TOT_TIME, 
    init_comps, N, **kwargs):    

  t_list = np.arange(0, TOT_TIME, DELTA_TIME)
  comps_lists = [[] for init in init_comps]

  num_of_comps = len(init_comps)
  if (calc_mode == "Approximate"):
    comps = init_comps
    for t in t_list:
      for i in range(num_of_comps):
        comps_lists[i].append(comps[i])
      comps_diffs = diff_eqs(comps, 0, N, **kwargs)
      for i in range(num_of_comps):
        comps[i] += comps_diffs[i] * DELTA_TIME
  else:
    init_X = np.array(init_comps)
    consts = (N, *kwargs.values())
    X = integrate.odeint(diff_eqs, init_X, t_list, args=consts)
    comps_lists = X.T

  print("Maximum Infectious population at a time :{}% \n".format(round(np.array(comps_lists[inf_index]).max()*100/N, 2)))

  plt.figure(figsize=(16,8))
  if (plot_type == "Lineplots"):
    for i in range(num_of_comps):
      plt.plot(t_list, comps_lists[i], label=comps_labels[i])
  else:
    y = np.vstack(comps_lists)
    plt.stackplot(t_list, y, labels=comps_labels)

  plt.xlabel("Time")
  plt.ylabel("Population " + ("Count" if N > 1 else "Percentage"))
  plt.legend()
  plt.show()

### SIR Model
$$
\begin{aligned}
\frac{dS}{dt} & = -\frac{\beta SI}{N} \\
\frac{dI}{dt} & = 
  \frac{\beta SI}{N} -\gamma I \\
\frac{dR}{dt} & = \gamma I
\end{aligned}
$$

In [ ]:
# nbi:hide_in
def sir_model(calc_mode, plot_type, 
    DELTA_TIME=.1, TOT_TIME=60,
    init_I_percent=.01, N=1,
    beta=.6, #Infection rate
    gamma=.1): #Recovery rate

  comps_labels = ("Susceptible", "Infected", "Recovered")
  inf_index = 1

  def sir_model_diff_eqs(comps, t, N, beta, gamma):
    S, I, R = comps
    return [
      -beta * S * I/N,
      beta * S * I/N - gamma * I,
      gamma * I ]

  S = N * (1 - init_I_percent)
  I = N * init_I_percent
  R = 0

  epid_model(comps_labels, inf_index, sir_model_diff_eqs, 
    calc_mode, plot_type, DELTA_TIME, TOT_TIME, 
    [S, I, R], N, beta=beta, gamma=gamma)

interact(sir_model, 
  calc_mode=["Approximate", "Solve with scipy"],
  plot_type=["Lineplots", "Stackplot"],
  DELTA_TIME=(.01, 1, .01), TOT_TIME=(0,100),  
  init_I_percent=(0.0, 1.0), N=(1, 1000), beta=(0.0, 1.0), gamma=(0.0, 1.0))

interactive(children=(Dropdown(description='calc_mode', options=('Approximate', 'Solve with scipy'), value='Ap…

### SIR Model with vital dynamics:
$$
\begin{aligned}
\frac{dS}{dt} & = 
  \Lambda -\frac{\beta SI}{N} -\mu S \\
\frac{dI}{dt} & = 
  \frac{\beta SI}{N} -\gamma I -\mu I \\
\frac{dR}{dt} & = \gamma I -\mu R
\end{aligned}
$$

In [ ]:
# nbi:hide_in
def sir_vitals_model(calc_mode, plot_type, 
    DELTA_TIME=.1, TOT_TIME=20,
    init_I_percent=.01, N=1,
    Lambda=10, #Birth rate
    mu=.0001, #Death rate
    beta=.6, #Infection rate
    gamma=.1): #Recovery rate

  comps_labels = ("Susceptible", "Infected", "Recovered")
  inf_index = 1

  def sir_model_vitals_diff_eqs(comps, t, N, Lambda, mu, beta, gamma):
    S, I, R = comps
    return [
      Lambda -beta * S * I/N -mu * S,
      beta * S * I/N - gamma * I -mu * I,
      gamma * I -mu * R]

  S = N * (1 - init_I_percent)
  I = N * init_I_percent
  R = 0

  epid_model(comps_labels, inf_index, sir_model_vitals_diff_eqs, 
    calc_mode, plot_type, DELTA_TIME, TOT_TIME, 
    [S, I, R], N, Lambda=Lambda, mu=mu, beta=beta, gamma=gamma)

interact(sir_vitals_model, 
  calc_mode=["Approximate", "Solve with scipy"],
  plot_type=["Lineplots", "Stackplot"],
  DELTA_TIME=(.01, 1, .01), TOT_TIME=(0,100),  
  init_I_percent=(0.0, 1.0), N=(1, 1000), 
  Lambda=(0, 1, .0001), mu=(0, 1, .0001), beta=(0.0, 1.0), gamma=(0.0, 1.0))

interactive(children=(Dropdown(description='calc_mode', options=('Approximate', 'Solve with scipy'), value='Ap…

### SIS Model:
$$
\begin{aligned}
\frac{dS}{dt} & = 
  -\frac{\beta SI}{N} +\gamma I \\
\frac{dI}{dt} & = 
  \frac{\beta SI}{N} -\gamma I\\
\end{aligned}
$$

In [ ]:
# nbi:hide_in
def sis_model(calc_mode, plot_type, 
    DELTA_TIME=.1, TOT_TIME=60,
    init_I_percent=.01, N=1,
    mu=.0001, #Death rate
    beta=.6, #Infection rate
    gamma=.1): #Recovery rate

  comps_labels = ("Susceptible", "Infected")
  inf_index = 1

  def sis_model_diff_eqs(comps, t, N, beta, gamma):
    S, I = comps
    return [
      -beta * S * I/N +gamma * I,
      beta * S * I/N -gamma * I]

  S = N * (1 - init_I_percent)
  I = N * init_I_percent

  epid_model(comps_labels, inf_index, sis_model_diff_eqs, 
    calc_mode, plot_type, DELTA_TIME, TOT_TIME, 
    [S, I], N, beta=beta, gamma=gamma)

interact(sis_model, 
  calc_mode=["Approximate", "Solve with scipy"],
  plot_type=["Lineplots", "Stackplot"],
  DELTA_TIME=(.01, 1, .01), TOT_TIME=(0,100),  
  init_I_percent=(0.0, 1.0), N=(1, 1000), 
  beta=(0.0, 1.0), gamma=(0.0, 1.0))

interactive(children=(Dropdown(description='calc_mode', options=('Approximate', 'Solve with scipy'), value='Ap…

### Vaccination of newborns and non-newborns:
$$
\begin{aligned}
\frac{dS}{dt} & = 
  \mu N(1-P) -\rho S 
  -\frac{\beta SI}{N} -\mu S \\
\frac{dI}{dt} & = 
  \frac{\beta SI}{N} -(\gamma + \mu)I \\
\frac{dV}{dt} & = \mu NP +\rho S  -\mu V\\
\end{aligned}
$$

In [ ]:
# nbi:hide_in
def sirv_model(calc_mode, plot_type, 
    DELTA_TIME=.1, TOT_TIME=60,
    init_I_percent=.01, N=1,
    # Lambda=10, #Birth rate
    mu=.0001, #Death rate
    P=.2, #Newborn vaccination rate
    rho=.1, #Non-newborn vaccination rate
    beta=.6, #Infection rate
    gamma=.1): #Recovery rate

  comps_labels = ("Susceptible", "Infected", "Vaccinated")
  inf_index = 1

  def sirv_model_diff_eqs(comps, t, N, mu, P, rho, beta, gamma):
    S, I, V = comps
    return [
      mu * N * (1 - P) -rho * S -beta * S * I/N -mu * S,
      beta * S * I/N -(gamma + mu) * I,
      mu * N * P + rho * S -mu * V ]

  S = N * (1 - init_I_percent)
  I = N * init_I_percent
  V = 0

  epid_model(comps_labels, inf_index, sirv_model_diff_eqs, 
    calc_mode, plot_type, DELTA_TIME, TOT_TIME, 
    [S, I, V], N, mu=mu, P=P, rho=rho, beta=beta, gamma=gamma)

interact(sirv_model, 
  calc_mode=["Approximate", "Solve with scipy"],
  plot_type=["Lineplots", "Stackplot"],
  DELTA_TIME=(.01, 1, .01), TOT_TIME=(0,100),  
  init_I_percent=(0.0, 1.0), N=(1, 1000), 
  mu=(0, 1, .0001), P=(0, 1, .0001), rho=(0, 1, .0001), beta=(0.0, 1.0), gamma=(0.0, 1.0))

interactive(children=(Dropdown(description='calc_mode', options=('Approximate', 'Solve with scipy'), value='Ap…

### SIRD Model:
$$
\begin{aligned}
\frac{dS}{dt} & = -\frac{\beta SI}{N} \\
\frac{dI}{dt} & = 
  \frac{\beta SI}{N} -\gamma I -\mu I \\
\frac{dR}{dt} & = \gamma I \\
\frac{dD}{dt} & = \mu I
\end{aligned}
$$

In [ ]:
# nbi:hide_in
def sird_model(calc_mode, plot_type, 
    DELTA_TIME=.1, TOT_TIME=60,
    init_I_percent=.01, N=1,
    mu=.001, #Death rate
    beta=.6, #Infection rate
    gamma=.1): #Recovery rate

  comps_labels = ("Susceptible", "Infected", "Recovered", "Deceased")
  inf_index = 1

  def sird_model_diff_eqs(comps, t, N, mu, beta, gamma):
    S, I, R, D = comps
    return [
      -beta * S * I/N,
      beta * S * I/N -gamma * I -mu * I,
      gamma * I,
      mu * I]

  S = N * (1 - init_I_percent)
  I = N * init_I_percent
  R = 0
  D = 0

  epid_model(comps_labels, inf_index, sird_model_diff_eqs, 
    calc_mode, plot_type, DELTA_TIME, TOT_TIME, 
    [S, I, R, D], N, mu=mu, beta=beta, gamma=gamma)

interact(sird_model, 
  calc_mode=["Approximate", "Solve with scipy"],
  plot_type=["Lineplots", "Stackplot"],
  DELTA_TIME=(.01, 1, .01), TOT_TIME=(0,100),  
  init_I_percent=(0.0, 1.0), N=(1, 1000), 
  mu=(0, 1, .0001), beta=(0.0, 1.0), gamma=(0.0, 1.0))

interactive(children=(Dropdown(description='calc_mode', options=('Approximate', 'Solve with scipy'), value='Ap…

### SEIR Model:
$$
\begin{aligned}
\frac{dS}{dt} & = 
  \mu N -\mu S -\frac{\beta SI}{N} \\
\frac{dE}{dt} & = 
  \frac{\beta SI}{N} -(\mu + a)E \\
\frac{dI}{dt} & = 
  aE -(\gamma + \mu)I \\
\frac{dR}{dt} & = \gamma I -\mu R\\
\end{aligned}
$$

In [ ]:
# nbi:hide_in
def seir_model(calc_mode, plot_type, 
    DELTA_TIME=.1, TOT_TIME=60,
    init_I_percent=.01, N=1,
    mu=.001, #Death rate
    beta=.6, #Infection rate
    a=.001, #Average incubation period
    gamma=.1): #Recovery rate

  comps_labels = ("Susceptible", "Exposed", "Infected", "Recovered")
  inf_index = 1

  def seir_model_diff_eqs(comps, t, N, mu, beta, a, gamma):
    S, E, I, R = comps
    return [
      mu * N -mu * S -beta * S * I/N,
      beta * S * I/N -(mu + a) * E,
      a * E -(gamma + mu) * I,
      gamma * I -mu * R ]

  S = N * (1 - init_I_percent)
  E = 0
  I = N * init_I_percent
  R = 0

  epid_model(comps_labels, inf_index, seir_model_diff_eqs, 
    calc_mode, plot_type, DELTA_TIME, TOT_TIME, 
    [S, E, I, R], N, mu=mu, beta=beta, a=a, gamma=gamma)

interact(seir_model, 
  calc_mode=["Approximate", "Solve with scipy"],
  plot_type=["Lineplots", "Stackplot"],
  DELTA_TIME=(.01, 1, .01), TOT_TIME=(0,100),  
  init_I_percent=(0.0, 1.0), N=(1, 1000), 
  mu=(0, 1, .0001), beta=(0.0, 1.0), a=(0, 1, .0001), gamma=(0.0, 1.0))

interactive(children=(Dropdown(description='calc_mode', options=('Approximate', 'Solve with scipy'), value='Ap…

### Zombie Apocalypse!:
(Variation of the SIR Model with vital dynamics)
$$
\begin{aligned}
\frac{dS}{dt} & = \Pi -\beta SZ -\delta S \\
\frac{dZ}{dt} & = \beta SZ +\zeta R -\alpha SZ \\
\frac{dR}{dt} & = \delta S +\alpha SZ -\zeta R
\end{aligned}
$$

In [ ]:
# nbi:hide_in
def solve_zombie_apoc(plot_data, plot_type, MAX_TIME=50, 
    init_S=100, init_Z=0, init_R=0,
    Pi=10, # Birth rate of susceptibles
    delta=.0001, # Death rate of susceptibles
    beta=.01, # Rate a susceptible becomes a zombie
    zeta=.0025, # Rate a dead person becomes a zombie
    alpha=.001): # Rate a zombie becomes destroyed

  def zombie_apoc_diff_eqs(S_Z_R, t=0):
    S, Z, R = S_Z_R
    return [
      Pi - beta * S * Z - delta * S,
      beta * S * Z + zeta * R - alpha * S * Z,
      delta * S + alpha * S * Z - zeta * R]
  
  t = np.linspace(0, MAX_TIME, int(250*MAX_TIME))
  init_X = np.array([init_S, init_Z, init_Z])
  X = integrate.odeint(zombie_apoc_diff_eq, init_X, t)
  S, Z, R = X.T

  fig = plt.figure(figsize=(16,8))
  if (plot_type == "Lineplots"):
    plt.plot(t, S, label="Living", color="tab:blue")
    plt.plot(t, Z, label="Zombies", color="tab:green")
    if (plot_data != "Living vs Zombies"):
      plt.plot(t, R, label="Decesased", color="tab:grey")
  else:
    stack = [S, Z]
    labels = ["Living", "Zombies"]
    colors = ["tab:blue", "tab:green"]
    if (plot_data != "Living vs Zombies"):
      stack.append(R)
      labels.append("Dceased")
      colors.append("tab:grey")
    y = np.vstack(stack)
    plt.stackplot(t, y, labels=labels, colors=colors)

  plt.xlabel("Time")
  plt.ylabel("Population Count")
  plt.legend(loc=2)
  plt.show()

interact(solve_zombie_apoc, 
  plot_data=["Living vs Zombies", "Living vs Zombies vs Deceased"],
  plot_type=["Lineplots", "Stackplot"],
  MAX_TIME=(0, 1000), init_S=(0, 1000), init_Z=(0, 1000), init_R=(0, 1000),
  Pi=(0, 1, .0001), delta=(0, 1, .0001), 
  beta=(0, 1, .0001), zeta=(0, 1, .0001), alpha=(0, 1, .0001))